# Programação orientada a objetos

Este notebook é a terceira parte da série sobre Programação orientada a objetos.

---
Nos notebooks anteriores, falamos sobre o primeiro pilar da POO: abstração/encapsulamento.

Neste notebook e no próximo, discutiremos os outros dois pilares que estão fortemente relacionados: herança e polimorfismo.

**DISCLAIMER: este notebook aborda esses dois conceitos de forma bastante tradicional. O próximo notebook usa uma visão revisada desses conceitos, principalmente sob a ótica de Python.**

## Herança

Herança é o mecanismo que permite que uma classe derivada reaproveite o código de uma classe base.

Em Python, dizemos que uma classe deriva (herda) de outra(s) listando as classes-base na definição da classe:

In [1]:
from collections import UserList
class Stack(UserList):
    def push(self, value):
        self.append(value)
    
    def pop(self):
        super().pop(-1)
    
    def top(self):
        return self.__getitem__(-1)

In [2]:
pilha = Stack()

In [3]:
pilha.push(3)
pilha.push(5)
pilha.push(10)
pilha.push(6)

In [4]:
pilha.top()

6

In [5]:
pilha.pop()
pilha.pop()
pilha.top()

5

Vamos discutir algumas coisas sobre esse código:

* A classe derivada herda acesso aos atributos e métodos da classe base. É possível acessá-los diretamente (como no método `push`) ou através da referência à classe base retornada pelo método embutido `super()` (como no método `pop`). A prioridade de resolução de escopo segue a ordem: local, classe, super, global.

* Como herdamos da classe `UserList`, não temos como acessar diretamente seus atributos por não sabermos como ela é implementada internamente. É por isso que tentamos usar sempre seus métodos, incluindo o método mágico `__getitem__` para acessar o último elemento no método `top`. 

**Obs #1: Também poderíamos acessar o último elemento usando `self[-1]`, uma vez que o operador `[]` invoca o método mágico `__getitem__`. Usei o método mágico explicitamente apenas para mostrar que podemos fazer isso :)**  

**Obs #2: Como a classe `UserList` foi projetada para ser extensível, ela expõe um atributo `data` para abstrair os dados que ela armazena. Assim, `self[-1]` e `self.data[-1]` são equivalentes.**

* A classe derivada pode reimplementar algum método existente na classe base. Chamamos isso de **sobrescrita de métodos**. Foi o que fizemos com o método `pop()`, que é implementado tanto por `UserList` como por `Stack`.

* Como Python não implementa herança privada, todos os métodos da classe base ficam expostos na classe derivada. Assim, a ordem da nossa pilha poderia ser alterada pelo usuário:

In [6]:
pilha.insert(1, 4) 

In [7]:
print(pilha)

[3, 4, 5]


### Herança múltipla

Python também permite que uma classe derivada herde métodos e atributos de múltiplas classes.

Como vamos discutir mais da frente, a herança simples é uma ferramenta que deve ser usada com cautela.

A herança múltipla, então, nem se fala 😂

Vamos ver um exemplo de herança múltipla e como o interpretador Python procede quando uma classe herda de classes base que implementam um método de mesmo nome: 

In [9]:
class Student():
    def __init__(self, student_id):
        self.student_id = student_id

    def __str__(self):
        return f"Student #ID: {self.student_id}"

class Employee():
    def __init__(self, employee_id):
        self.employee_id = employee_id

    def __str__(self):
        return f"Employee #ID: {self.employee_id}"

class Trainee(Student, Employee):
    def __init__(self, student_id, employee_id):
        Student.__init__(self, student_id)
        Employee.__init__(self, employee_id)

In [10]:
leo = Trainee(5534, 4529)
print(leo)

Student #ID: 5534


Note que, apesar de ser possível usar o procedimento `super()`, faz pouco sentido fazer isso considerando que temos múltiplas classes-base.

Em vez disso, invocamos os métodos `__init__` de cada classe base e precisamos passar a referência `self` para esses métodos.

Além disso, a chamada ao procedimento `print()` acessa apenas o método mágico `__str__` da classe `Student`. Isto acontece porque o interpretador Python usa um algoritmo para determinar a precedência de métodos de mesmo nome presentes nas classes base herdadas.

O resultado deste algoritmo está presente no atributo de classe `__mro__` (do inglês, *method resolution order*) e varia em função ordem em que as classes-base são listadas na declaração de herança:

In [11]:
Trainee.__mro__

(__main__.Trainee, __main__.Student, __main__.Employee, object)

In [12]:
class Trainee(Employee, Student):
    def __init__(self, student_id, employee_id):
        Student.__init__(self, student_id)
        Employee.__init__(self, employee_id)

In [13]:
Trainee.__mro__

(__main__.Trainee, __main__.Employee, __main__.Student, object)

Neste caso, é simples acessar ambos os métodos `__str__`, fazendo uma sobrescrita deste método na classe `Trainee`:

In [14]:
class Trainee(Employee, Student):
    def __init__(self, student_id, employee_id):
        Student.__init__(self, student_id)
        Employee.__init__(self, employee_id)
        
    def __str__(self):
        return "\n".join([Student.__str__(self), Employee.__str__(self)])

In [15]:
leo = Trainee(5534, 4529)
print(leo)

Student #ID: 5534
Employee #ID: 4529


Também é possível usar a MRO para acessar todos os métodos de mesmo nome das classes-base de `Trainee`:

In [16]:
class Trainee(Employee, Student):
    def __init__(self, student_id, employee_id):
        Student.__init__(self, student_id)
        Employee.__init__(self, employee_id)
        
    def __str__(self):
        return "\n".join([base.__str__(self) for base in type(self).__mro__[1:-1]])

In [17]:
leo = Trainee(5534, 4529)
print(leo)

Employee #ID: 4529
Student #ID: 5534


## Polimorfismo

Python é uma linguagem dinâmica que se baseia no polimorfismo de maneira bastante natural.

Os containers embutidos são um forte exemplo de **polimorfismo paramétrico**, que em outras linguagens é implementado através de programação genérica (e.g., templates em C++):

In [17]:
print([10, 3, 7, 5])
print({0.4, 7.6, 3.4})
print({"a": "c", "e": "b"})

[10, 3, 7, 5]
{0.4, 3.4, 7.6}
{'a': 'c', 'e': 'b'}


De fato, a capacidade polimórfica dos objetos Python é ainda maior do que a de linguagens com tipagem estática como C++ ou Java. 

O exemplo abaixo mostra a capacidade dos containers Python para armazenar tipos heterogêneos de dados de forma dinâmica, o que não é possível diretamente nessas outras linguagens:

In [18]:
hetero_set = set()

In [19]:
hetero_set.add(5)
hetero_set.add("a")
hetero_set.add(3.4)
hetero_set.add((3))

In [20]:
print(hetero_set)
print("a" in hetero_set)
print("b" in hetero_set)
print(4.1 in hetero_set)
print(3 in hetero_set)

{3.4, 3, 'a', 5}
True
False
False
True


Dado o elevado polimorfismo inerente a linguagem, nossa postura tem que ser de aproveitar ao máximo a integração com o repertório de tipos e procedimentos polimórficos disponíveis na linguagem.

Pra entender as opções de polimorfismo existentes em Python, vamos começar discutindo interfaces e no notebook seguinte discutiremos protocolos.

### Interfaces

Linguagens de programação mais tradicionais adotam o conceito de interfaces (ou, mais geralmente, classes abstratas), que ajudam a formalizar a característica de um conjunto de classes.

O módulo `collections.abc` traz vários exemplos de interfaces, como a interface `Sized`, que indica que um objeto consegue determinar/informar seu tamanho:

In [18]:
from collections.abc import Sized
print(isinstance(list(), Sized))
print(isinstance(set(), Sized))
print(isinstance(str(), Sized))

True
True
True


Dizemos que uma classe implementa uma interface quando ela herda da interface e implementa os métodos prescritos na interface.

No caso da interface `Sized`, o método prescrito é o método mágico `__len__`.

Vamos fazer uma classe `Team` que armazena uma lista de jogadores de uma equipe. Vamos garantir que ela implemente a interface `Sized`: 

In [19]:
class Team(Sized):
    def __init__(self, args=[]):
        self.players = list(args)

In [20]:
equipe1 = Team()

TypeError: Can't instantiate abstract class Team with abstract methods __len__

**Note que o interpretador Python chama a interface `Sized` de classe abstrata. Mais na frente vamos discutir as nuances que diferenciam esses dois conceitos.**

Apenas herdar da interface `Sized` não é suficiente, porque o método `__len__` prescrito nessa interface é um **método abstrato**.

Isto quer dizer que a interface `Sized` especifica que todas as suas classes derivadas precisam implementar o método mágico `__len__`, mas não fornece uma implementação padrão.

Vamos completar nossa classe `Team` pra que ela implemente a interface `Sized`:

In [21]:
class Team(Sized):
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)

In [22]:
equipe1 = Team()
len(equipe1)

0

In [23]:
equipe2 = Team(["Python", "C++", "Ruby", "Java"])
len(equipe2)

4

### Implementando múltiplas interfaces

Interfaces são um exemplo onde herança múltipla é útil. Vamos adicionar mais uma característica à nossa classe `Team`, implementando a interface `Container`:

In [24]:
from collections.abc import Container
class Team(Sized, Container):
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)

In [25]:
equipe1 = Team()

TypeError: Can't instantiate abstract class Team with abstract methods __contains__

Note que o método prescrito pela interface `Container` é o método mágico `__contains__`, que usamos através do operador `in`.

In [26]:
from collections.abc import Container
class Team(Sized, Container):
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)
    
    def __contains__(self, value):
        return value in self.players

In [27]:
equipe1 = Team()
print("Leo" in equipe1)

False


In [28]:
equipe2 = Team(["Python", "C++", "Ruby", "Java"])
print("Python" in equipe2)

True


Em alguns casos, a combinação de interfaces forma um conceito mais amplo, também fornecido como interface.

É o caso da interface `Sequence`, que agrega as funcionalidades das interfaces `Sized`, `Container` e `Iterable`:

In [29]:
from collections.abc import Sequence
class Team(Sequence):
    def __init__(self, args=[]):
        self.players = list(args)

In [30]:
equipe1 = Team()

TypeError: Can't instantiate abstract class Team with abstract methods __getitem__, __len__

A interface `Sequence` prescreve os métodos `__getitem__` e `__len__`. Vamos adicioná-los à nossa classe:

In [31]:
from collections.abc import Sequence
class Team(Sequence):
    def __init__(self, args=[]):
        self.players = list(args)
    
    def __len__(self):
        return len(self.players)
    
    def __getitem__(self, index):
        return self.players[index]

In [32]:
equipe2 = Team(["Python", "C++", "Ruby", "Java"])

In [33]:
for i in range(len(equipe2)):
    print(equipe2[i])

Python
C++
Ruby
Java


Note que dissemos que a interface `Sequence` agrega as funcionalidades das interfaces `Sized`, `Container` e `Iterable`, mas nós não chegamos a implementar os métodos prescritos em `Container` (método mágico `__contains__`) nem em `Iterable` (método mágico `__iter__`).

No entanto, nós conseguimos usar os dois métodos na nossa classe `Team`:

In [34]:
"Ruby" in equipe2

True

In [35]:
for lang in equipe2:
    print(lang)

Python
C++
Ruby
Java


Isto acontece porque a interface `Sequence` implementa os métodos `__contains__` e `__iter__` a partir do método `__getitem__`.

Pra ser mais preciso, podemos distinguir três conceitos importantes relacionados a polimorfismo:
* **interfaces** prescrevem métodos
* **classes-base abstratas (ABCs, do inglês *abstract base classes*)** prescrevem e/ou fornecem métodos
* **mixins** fornecem métodos

Alguns comentários sobre isso:
- Estávamos chamando `Sequence` de interface, mas agora vemos que o correto é chamá-la de ABC.
- Toda interface e mixin são ABCs, mas a recíproca não é necessariamente verdadeira.
- A documentação do módulo `collections.abc` do Python considera que um *mixin* é um método que uma classe ganha de brinde quando implementa uma classe-base abstrata.

### Mixins e herança múltipla



Interfaces devem ser pensadas de forma a casar com a necessidade do cliente. Este é um dos cinco princípios básicos que ajudam a nortear o processo de modelagem de um sistema orientado a objetos, conhecidos pelo acrônimo [S.O.L.I.D.](https://scotch.io/bar-talk/s-o-l-i-d-the-first-five-principles-of-object-oriented-design):

> A client should never be forced to implement an interface that it doesn't use or clients shouldn't be forced to depend on methods they do not use.

Tradução livre:
> "Um cliente nunca deveria ser forçado a implementar uma interface que ele não use" ou "clientes não devem ser forçados a depender de métodos que eles não usem".

Este pode ser o caso de um cliente que decida implementar a ABC `Sequence`, mas não tenha interesse no método `__contains__`, ou não tenha como implementar o método `__len__`.

É por isso que **mixins** se tornaram populares em linguagens que fornecem polimorfismo através de ABCs. 

Uma mixin bem projetada costuma fornecer apenas uma funcionalidade, então herdar de várias mixins dificilmente se torna um problema. Além disso, só faz sentido implementar mixins que representem comportamentos comuns o suficiente para serem reutilizado por várias classes.

> Mixins take various forms depending on the language, but at the end of the day they encapsulate behavior that can be reused in other classes.

[Fonte](https://easyaspython.com/mixins-for-fun-and-profit-cb9962760556). Tradução livre:

> Mixins podem assumir diferentes formas depedendo da linguagem, mas no fim das contas elas encapsulam comportamento que pode ser reusado em outras classes.

O exemplo a seguir mostra duas mixins que permitem serializar objetos nos formatos JSON e Pickle. Por convenção, acrescentamos o sufixo Mixin ao nome do mixin:

In [42]:
from collections import UserDict

class JSONExporterMixin:
    def json(self):
        from json import dumps
        return dumps(self.data)

class PickleExporterMixin:
    def pickle(self):
        from pickle import dumps
        return dumps(self.data)
    
class SerialDict(JSONExporterMixin, PickleExporterMixin, UserDict):
    pass

In [43]:
teste = SerialDict({"a": 345, "b": 648})

In [44]:
teste.json()

'{"a": 345, "b": 648}'

In [45]:
teste.pickle()

b'\x80\x03}q\x00(X\x01\x00\x00\x00aq\x01MY\x01X\x01\x00\x00\x00bq\x02M\x88\x02u.'

Usando a mesma lógica, nós podemos fazer mixins que forneçam fábricas para construir dicionários customizados a partir de strings em formato JSON ou Pickle:

In [46]:
class JSONImporterMixin:
    @classmethod
    def from_json(cls, json):
        from json import loads
        return cls(loads(json))

class PickleImporterMixin:
    @classmethod
    def from_pickle(cls, pickle):
        from pickle import loads
        return cls(loads(pickle))
    
class SerialDict(JSONImporterMixin, PickleImporterMixin, JSONExporterMixin, PickleExporterMixin, UserDict):
    pass

In [47]:
teste = SerialDict({"a": 345, "b": 648})

In [48]:
json = teste.json()

In [49]:
teste2 = SerialDict.from_json(json)

In [50]:
teste == teste2

True

In [51]:
pickle = teste.pickle()

In [52]:
teste2 = SerialDict.from_pickle(pickle)

In [53]:
teste == teste2

True